In [ ]:
# offline_train_optimized.py

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models


data_dir = r"D:\demmo\plant\data"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "test")
batch_size = 16             # smaller batch on CPU, increase on GPU
num_epochs = 15
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(25),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

num_classes = len(train_dataset.classes)
print(f"Classes ({num_classes}): {train_dataset.classes}")


model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Freeze feature extractor
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.classifier.parameters(), lr=learning_rate)  # only train classifier
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


best_acc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

   
    model.train()
    running_loss, running_corrects = 0.0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    train_loss = running_loss / len(train_dataset)
    train_acc = running_corrects.double() / len(train_dataset)

  
    model.eval()
    val_loss_total, val_corrects = 0.0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_loss_total += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

    val_loss = val_loss_total / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)

    scheduler.step()

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), os.path.join(data_dir, "best_plantdoc_model.pth"))
        print(f"✅ Saved Best Model (Val Acc: {best_acc:.4f})")

print("\nTraining complete ✅")
print(f"Best Validation Accuracy: {best_acc:.4f}")


Using device: cpu
Classes (28): ['Apple_Scab_Leaf', 'Apple_leaf', 'Apple_rust_leaf', 'Bell_pepper_leaf', 'Bell_pepper_leaf_spot', 'Blueberry_leaf', 'Cherry_leaf', 'Corn_Gray_leaf_spot', 'Corn_leaf_blight', 'Corn_rust_leaf', 'Peach_leaf', 'Potato_leaf_early_blight', 'Potato_leaf_late_blight', 'Raspberry_leaf', 'Soyabean_leaf', 'Squash_Powdery_mildew_leaf', 'Strawberry_leaf', 'Tomato_Early_blight_leaf', 'Tomato_Septoria_leaf_spot', 'Tomato_leaf', 'Tomato_leaf_bacterial_spot', 'Tomato_leaf_late_blight', 'Tomato_leaf_mosaic_virus', 'Tomato_leaf_yellow_virus', 'Tomato_mold_leaf', 'Tomato_two_spotted_spider_mites_leaf', 'grape_leaf', 'grape_leaf_black_rot']
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\User/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100.0%



Epoch 1/15
Train Loss: 3.1602 | Train Acc: 0.1064 | Val Loss: 3.1566 | Val Acc: 0.1389
✅ Saved Best Model (Val Acc: 0.1389)

Epoch 2/15
Train Loss: 2.9159 | Train Acc: 0.2371 | Val Loss: 3.0267 | Val Acc: 0.1944
✅ Saved Best Model (Val Acc: 0.1944)

Epoch 3/15
Train Loss: 2.7474 | Train Acc: 0.3056 | Val Loss: 2.9088 | Val Acc: 0.2579
✅ Saved Best Model (Val Acc: 0.2579)

Epoch 4/15
Train Loss: 2.6071 | Train Acc: 0.3371 | Val Loss: 2.8106 | Val Acc: 0.2897
✅ Saved Best Model (Val Acc: 0.2897)

Epoch 5/15
Train Loss: 2.4913 | Train Acc: 0.3640 | Val Loss: 2.7320 | Val Acc: 0.3175
✅ Saved Best Model (Val Acc: 0.3175)

Epoch 6/15
Train Loss: 2.3990 | Train Acc: 0.3955 | Val Loss: 2.6770 | Val Acc: 0.3254
✅ Saved Best Model (Val Acc: 0.3254)

Epoch 7/15
Train Loss: 2.3288 | Train Acc: 0.4142 | Val Loss: 2.6544 | Val Acc: 0.3413
✅ Saved Best Model (Val Acc: 0.3413)

Epoch 8/15
Train Loss: 2.2763 | Train Acc: 0.4251 | Val Loss: 2.5891 | Val Acc: 0.3571
✅ Saved Best Model (Val Acc: 0.3571)
